# 1. Inicjalizacja API
W pierwszej komórce wskazujemy adres serwera cloudberry, a w drugiej tworzymy metadane opisujące eksperyment dla którego dane chcemy zaimportować.

In [ ]:
# Define Cloudberry configuration

import cloudberry.api as cb
cb_port = 9000
cb_config = cb.CloudberryConfig(f'http://localhost:{cb_port}')

In [ ]:
# Create metadata necessary for CSV upload - Experiment & Configuration

meta_api = cb.Metadata(cb_config)
experiment_api = meta_api.experiment_api()
configuration_api = meta_api.experiment_configuration_api()

csv_experiment_name = 'CSV Scenario Experiment'
csv_configuration_name = 'CSV Scenario Configuration'

# optionally pass parameters to save experiment with
csv_experiment = experiment_api.find_or_create(csv_experiment_name, parameters = {})
csv_configuration = configuration_api.find_or_create(csv_experiment, csv_configuration_name, parameters={'version': 1})

Właściwy import danych z przykładowego pliku CSV:

In [ ]:
# CSV-like Data Upload API

csv_api = cb.CsvFileUploader(cb_config)

csv_file_path = './data/csv_evolution_data.csv'
csv_details = cb.CsvUploadDetails(
    tags_names=['ISLAND_ID'],
    raw_fields=['BEST_SOLUTION'],
    configuration=csv_configuration,
    computation=None, # None for new one, otherwise append data to existing computation .
)

csv_computation = csv_api.upload_file(
    csv_file_path,
    csv_experiment_name,
    csv_details
)

Weryfikacja zaimportowanych danych:

In [ ]:
data_api = cb.Data(cb_config)
filters = cb.DataFilters(tags={}, computation=csv_computation)
series = data_api.get_data(filters)
series.as_data_frame

Wykres dla parametru BEST_MERIT_FACTOR:

In [ ]:
cb_computations = cb.ComputationSeries(cb_config)

field = 'BEST_MERIT_FACTOR_VALUE'
series_pack = cb_computations.for_computations([csv_computation], field)

In [ ]:
from cloudberry.converters import SeriesConverter

plot_pack = SeriesConverter.data_series_pack_to_plot_series_pack(
    ds=series_pack,
    x_field="_time",
    y_field=field,
)

In [ ]:
import cloudberry.plots as cp

props = cp.PlotProperties.default()
props.title = "BEST_MERIT_FACTOR_VALUE for uploaded computation from csv file"
props.show_title = True
props.x_axis_name = "Time in milliseconds"
props.y_axis_name = "BEST_MERIT_FACTOR_VALUE"
props.show_averages = False
# props.show_series = False
# props.show_error_bars = True
props.default_series_kind = cp.PlotSeriesKind.LINE

In [ ]:
builder = cp.PlotBuilder(props)
builder.add_pack(plot_pack)
plot = builder.plot()
plot